In [ ]:
from re import split
from fastai import *
from fastai.vision import *
from fastai.vision.all import *
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print(torch.cuda.is_available())

In [ ]:
path = Path('/home/qiao/dev/giao/datasets/kittiseg')

path_img = path/'training/image_2'
path_label = path/'training/instance'
codes = ['void', 'flat', 'construction', 'object', 'nature', 'sky', 'human', 'vehicle']
# codes = ['unlabeled',
#     'ego vehicle',
#     'rectification border',
#     'out of roi',
#     'static',
#     'dynamic',
#     'ground',
#     'road',
#     'sidewalk',
#     'parking',
#     'rail track',
#     'building',
#     'wall',
#     'fence',
#     'guard rail',
#     'bridge',
#     'tunnel',
#     'pole',
#     'polegroup',
#     'traffic light',
#     'traffic sign',
#     'vegetation',
#     'terrain',
#     'sky',
#     'person',
#     'rider',
#     'car',
#     'truck',
#     'bus',
#     'caravan',
#     'trailer',
#     'train',
#     'motorcycle',
#     'bicycle',
#     'license plate']

In [ ]:
path_img
path_label

In [ ]:
fnames = get_image_files(path_img)
print(fnames[0])

In [ ]:
lbl_names = get_image_files(path_label)
print(lbl_names[0])

In [ ]:
img = Image.open(fnames[0])
img.show()

In [ ]:
img = Image.open(lbl_names[0])
img.show()

In [ ]:
get_y_fn = lambda x: path_label/f'{x.name}'
defaults.use_cuda = True

In [ ]:
dls = SegmentationDataLoaders.from_label_func(
    path_img,
    bs = 1,
    fnames = fnames,
    label_func = get_y_fn,
    codes = codes,
    item_tfms = [Resize((400, 400))],
    batch_tfms = [Normalize.from_stats(*imagenet_stats)],
)

In [ ]:
name2id = {v:k for k, v in enumerate(codes)}
void_code = name2id['void']

def acc_smoke(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    # mask = target
    return (input.argmax(dim = 1)[mask]==target[mask]).float().mean()

metrics = acc_smoke

In [ ]:
learn = unet_learner(dls, models.resnet34, metrics = metrics,
                    self_attention=True)

In [ ]:
device = 'cuda'
learn.model.to(device)

In [ ]:
# learn.model

In [ ]:
learn.lr_find(stop_div=False, num_it=8)

In [ ]:
lr = 5e-4
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.recorder.plot_loss(skip_start=0, with_valid=True)

In [ ]:
learn.save('stage-1')
learn.load('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
lr_max = 1.73e-5

In [ ]:
learn.fit_one_cycle(4, lr_max)

In [ ]:
learn.recorder.plot_loss(skip_start=0, with_valid=True)

In [ ]:
learn.show_results()
learn.save('U-res34')

In [ ]:
# learn.show_results()
learn.load('U-res34')
learn.show_results()

In [ ]:
learn.show_results()

In [ ]:
learn.show_results()

In [ ]:
learn.show_results()
learn.save('alunet')

In [ ]:
learn.show_results()

In [ ]:
learn.show_results()

In [ ]:
learn.show_results()

In [ ]:
img = Image.open('/home/qiao/dev/giao/datasets/S_google_wildfire/004.png').resize((255, 255))
img.show()

In [ ]:
img = PILImage(img)

In [ ]:
learn.load('alunet')
# learn.load('attunet')

In [ ]:
prediction_mask = learn.predict(img)
seg_map = prediction_mask[0]
torch.Tensor.ndim = property(lambda self: len(self.shape))
plt.imshow(seg_map)